# CRF 实战

下述代码源自于pytorch官网教程：
https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

## CRF 回顾

CRF是一种常见的，建模序列化数据概率分布的一种判别式模型。其主要目的为建模序列数据间的条件概率分布：

$P(y_1,y_2,...,y_C | x_1, x_2, ..., x_C)$

CRF常见的参数化形式如下：

$P(y|x, w)=\frac{1}{Z(w,x)}exp\left[ \sum_{j=1}^{n} w_j \sum_{i=1}^{C}f_j(x, y_i, y_{i-1}) \right]$

其中$w_j$为模型参数，$f_j(x, y_i, y_{i-1})$为特征函数，$C$为序列长度，$n$为特征数量，$Z(w,x)$为归一化因子，由下式确定：

$Z(w,x)= \sum_{y'} exp\left[ \sum_{j=1}^{n} w_j \sum_{i=1}^{C}f_j(x, y_i, y_{i-1}) \right]$

上式中的求和符号会遍历所有可能的长度为$C$的标签序列。

常见的CRF文献中，会把特征函数区分为两类，一类是状态特征$s_l(y_i,x,i)$，即由某个标记和观测所确定。另一类是转移特征$t_k(y_{i-1},y_i,x,i)$，即由两个相邻的标记所确定：

$P(y|x, w)=\frac{1}{Z(w,x)}exp\left[ \sum_{i,k} \lambda_k t_k(y_{i-1},y_i,x,i) +  \sum_{i,l} \mu_l s_l(y_i,x,i) \right]$

上式中，$w=\lambda, \mu$ 为模型参数。

## LSTM + CRF

CRF在实际使用过程中，经常会与LSTM（或其他序列特征提取器，如BERT）结合使用。这种做法最早在论文“ Bidirectional LSTM-CRF Models for Sequence Tagging”中提出。

具体来说，我们会把LSTM模型所提取的特征输入到CRF模型中，并且在CRF模型中显示地建模了状态之间的转移规律。

假设我们模型中所建模的标记数量共有$m$个，我们的LSTM模型在每个时间步会输出一个$m$维的表示向量，我们会把这个向量作为上层CRF模型的状态特征$s_l(y_i,x,i)$，并在CRF模型中学习一个转移矩阵（转移特征的参数）。

下面让我们来看一下这些计算是如何实现的：

In [3]:
# Original Author: Robert Guthrie
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

def argmax(vec):
    # 取一个列表的argmax
    _, idx = torch.max(vec, 1)
    return idx.item()

# 将输入的词序列转换成id，可以使用tokenizer实现
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# 计算一个向量的log sum exp
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

构建模型

In [8]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        # 构建LSTM模型
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # 将LSTM模型的输入映射到self.tagset_size这么大
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # 定义转移矩阵，即各个转移特征所对应的参数
        # 矩阵中的元素 i,j 表示从标签 j 转移到标签 i 的分数.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # 将转移矩阵中 START 和 STOP 标签所对应的分数设成无穷小。
        # 从而保证不会转移到 START 状态，也不会从 STOP 转移出去
        # 所有状态序列都从 START 开始，从 STOP 结束
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # 在前向算法中计算归一化因子 Z(x, w)
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG 在初始时间步的分数最大
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # 为了利用pytorch实现梯度回传，我们创建一个 variable
        forward_var = init_alphas

        # 对于每一个位置 t
        for feat in feats:
            alphas_t = []  # alpha_t[i] = \alpha(t+1, y_i)，即在位置t+1的标记为y_i，并且从1到t的前部分标记序列的非规范化概率
            
            for next_tag in range(self.tagset_size):  # 对于每一个标记y_i
                
                # fest 为LSTM在时间步t的输出，即发射分数。与前一个位置的标记无关
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                
                # 转移矩阵的第i行指的是，从各个状态转移到next_tag (y_i)这个状态的转移分数
                trans_score = self.transitions[next_tag].view(1, -1)
                
                # 将转移分数和发射分数累加起来，计算各个状态转移到next_tag (y_i)的分数
                next_tag_var = forward_var + trans_score + emit_score
                
                # 计算\alpha(t+1, y_i)，即在位置t+1的标记为y_i，并且从1到t的前部分标记序列的非规范化概率.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
                
            forward_var = torch.cat(alphas_t).view(1, -1)
            
        # 计算 \alpha(t+1, STOP_TAG) 即最后一个位置是STOP_TAG的所有标记序列的非规范化概率
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # 给定输入序列和标记序列，求对应的分数
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):  # 对于序列中的每个位置
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]  # 累加当前位置的转移分数和发射分数
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]# 累加转移到STOP_TAG的转移分数 
        return score

    def _viterbi_decode(self, feats):
        backpointers = []  # 初始化维特比算法的中间变量

        # 初始化维特比算法的中间变量
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0 

        # forward_var[i]中的元素对应 0：i-1 非规范化概率最大值
        forward_var = init_vvars
        for feat in feats:  # 对于每个时间步 k
            bptrs_t = []    # 记录最大跳转位置
            viterbivars_t = []  # 记录下一个时间步 k+1 的维特比变量 u(k+1,V)

            for next_tag in range(self.tagset_size):  # 对于每个标签 V
                # next_tag_var[i] 表示 u(k,y_i) + 转移分数
                # 这里不考虑发射分数，因为发射分数和前一个时间步的标签无关
                next_tag_var = forward_var + self.transitions[next_tag]
                
                # 取最大转移位置，并记录在 bptrs_t 中
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                
                # 记录 u(k+1,y_i) - 发射分数
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            
            # 计算u(k+1,y_i)，也就是把发射分数加回来
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # 计算跳转到 STOP_TAG 的非规范化序列
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)  # 得到概率最大的跳转位置，即最后一个位置的标签
        path_score = terminal_var[0][best_tag_id]

        # 从最后一个位置的标签获得整个标签序列
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score  # 计算负指数损失

    def forward(self, sentence):  # 注意，这里是pytorch的module方法中应该定义的函数，和前向算法无关.
        # 从Bi-LSTM中获得发射分数
        lstm_feats = self._get_lstm_features(sentence)

        # 寻找最优路径
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [5]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# 假设有这些训练数据
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}  # 构建一个vocab
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# 在训练之前先看看模型的输出
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# 开始训练
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. 清空grad
        model.zero_grad()

        # Step 2. 准备模型输入
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. 计算loss，也就是负指数损失
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. 做反向传播
        loss.backward()
        optimizer.step()

# 训练后再检查一下模型输出
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

(tensor(2.6907), [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
(tensor(20.4906), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
